# Northwind Data Visualization - Interactive Dashboard

This notebook contains interactive visualizations using Plotly, including delivery statistics and 3D analysis.

In [13]:
# Install required packages if not already installed
!pip install plotly nbformat pandas --quiet

In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Ensure figures dir exists
os.makedirs("../figures", exist_ok=True)

In [15]:
# Load the extracted data from CSVs
extracted_dir = "../data/extracted"

try:
    fact_orders = pd.read_csv(os.path.join(extracted_dir, "FactOrders.csv"))
    dim_customers = pd.read_csv(os.path.join(extracted_dir, "DimCustomer.csv"))
    dim_employees = pd.read_csv(os.path.join(extracted_dir, "DimEmployee.csv"))
    dim_date = pd.read_csv(os.path.join(extracted_dir, "DimDate.csv"))

    # Merge DataFrames to recreate the analysis dataset
    # 1. Join Orders with Customers
    df = fact_orders.merge(dim_customers, on="CustomerId", how="left")
    
    # 2. Join with Employees (will create _x (Customer) and _y (Employee) suffixes)
    df = df.merge(dim_employees, on="EmployeeId", how="left")
    
    # 3. Join with Date
    df = df.merge(dim_date, on="DateId", how="left")

    df['FullDate'] = pd.to_datetime(df['FullDate'])
    print("Data loaded and merged successfully from new CSVs.")
    print(f"Total records: {len(df)}")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please run the extraction scripts first.")

Data loaded and merged successfully from new CSVs.
Total records: 48
   OrderId  CustomerId  EmployeeId    DateId  DeliveredFlag CompanyName  \
0       30          27           9  20060115              1  Company AA   
1       31           4           3  20060120              1   Company D   
2       32          12           4  20060122              1   Company L   
3       33           8           6  20060130              1   Company H   
4       34           4           9  20060206              1   Company D   

      City_x Country_x FirstName        LastName    City_y Country_y  \
0  Las Vegas       USA      Anne  Hellung-Larsen   Seattle       USA   
1   New York       USA       Jan           Kotas   Redmond       USA   
2  Las Vegas       USA    Mariya       Sergienko  Kirkland       USA   
3   Portland       USA   Michael         Neipper   Redmond       USA   
4   New York       USA      Anne  Hellung-Larsen   Seattle       USA   

    FullDate  Day  Month MonthName  
0 2006-01-

## 1. Delivery Status Overview

In [16]:
if 'df' in locals():
    delivery_counts = df['DeliveredFlag'].value_counts()
    
    fig = go.Figure(data=[go.Pie(
        labels=['Delivered', 'Not Delivered'],
        values=[delivery_counts.get(1, 0), delivery_counts.get(0, 0)],
        hole=0.3,
        marker=dict(colors=['#10b981', '#f43f5e']),
        textinfo='label+percent+value',
        hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
    )])
    
    fig.update_layout(
        title='Order Delivery Status',
        font=dict(size=14),
        height=500
    )
    
    try:
        fig.write_html("../figures/delivery_stats_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 2. Orders by Country (Interactive)

In [17]:
if 'df' in locals():
    country_orders = df['Country_x'].value_counts().reset_index()
    country_orders.columns = ['Country', 'OrderCount']
    
    fig = px.bar(
        country_orders,
        x='Country',
        y='OrderCount',
        title='Total Orders by Country',
        color='OrderCount',
        color_continuous_scale='Tealgrn',
        hover_data={'OrderCount': ':,'}
    )
    
    fig.update_layout(
        xaxis_title='Country',
        yaxis_title='Number of Orders',
        height=600
    )
    
    try:
        fig.write_html("../figures/orders_by_country_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 3. Orders by Employee (Interactive)

In [18]:
if 'df' in locals():
    df['EmployeeName'] = df['FirstName'] + ' ' + df['LastName']
    employee_orders = df['EmployeeName'].value_counts().reset_index()
    employee_orders.columns = ['EmployeeName', 'OrderCount']
    
    fig = px.bar(
        employee_orders,
        y='EmployeeName',
        x='OrderCount',
        orientation='h',
        title='Orders by Employee',
        color='OrderCount',
        color_continuous_scale='Purples'
    )
    
    fig.update_layout(
        xaxis_title='Number of Orders',
        yaxis_title='Employee',
        height=600
    )
    
    try:
        fig.write_html("../figures/orders_by_employee_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 4. Monthly Orders Trend (Interactive)

In [19]:
if 'df' in locals():
    df['YearMonth'] = df['FullDate'].dt.to_period('M').astype(str)
    monthly_orders = df.groupby('YearMonth').size().reset_index(name='OrderCount')
    
    fig = px.line(
        monthly_orders,
        x='YearMonth',
        y='OrderCount',
        title='Monthly Orders Trend',
        markers=True
    )
    
    fig.update_traces(
        line=dict(color='#6366f1', width=3),
        marker=dict(size=8)
    )
    
    fig.update_layout(
        xaxis_title='Month',
        yaxis_title='Number of Orders',
        height=600,
        hovermode='x unified'
    )
    
    try:
        fig.write_html("../figures/monthly_trend_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 5. 3D Interactive Visualization: Orders by Month and Country

In [20]:
# --- MOCK DATA GENERATION (1996-2005) ---
# The source data only contains 2006. We generate mock history for visualization purposes.
import numpy as np

if 'df' in locals():
    print("Generating mock historical data (1996-2005)...")
    
    # Get existing unique entities to maintain referential consistency
    customers = df['CompanyName'].unique()
    employees = df['LastName'].unique()
    
    mock_rows = []
    # Generate ~500 mock orders scattered across the years
    for _ in range(500):
        # Random Year between 1996 and 2005
        year = np.random.randint(1996, 2006)
        # Random Month and Day
        month = np.random.randint(1, 13)
        day = np.random.randint(1, 28)
        mock_date = pd.Timestamp(year=year, month=month, day=day)
        
        row = {
            'CompanyName': np.random.choice(customers),
            'LastName': np.random.choice(employees),
            'FullDate': mock_date,
            'Year': year,
            'OrderCount': 1, # Placeholder
            'DeliveredFlag': np.random.choice([0, 1]), # Add this for delivery stats
            'Country_x': 'USA' # Default mock country to avoid NaN in country stats
        }
        mock_rows.append(row)
    
    mock_df = pd.DataFrame(mock_rows)
    
    # Ensure original df has 'Year' computed if not already
    if 'Year' not in df.columns:
        df['FullDate'] = pd.to_datetime(df['FullDate'])
        df['Year'] = df['FullDate'].dt.year
    
    # FIXED LINE: Do not subset df. Concatenate everything. 
    # Mock data will have NaNs for columns we didn't mock, which is fine.
    df = pd.concat([df, mock_df], ignore_index=True)
    
    print(f"Added {len(mock_df)} mock records. Total records: {len(df)}")


Generating mock historical data (1996-2005)...
Added 500 mock records. Total records: 548


In [21]:
# OLAP 3D Visualization: Orders by Customer, Employee, Date (with Year Selection)
if 'df' in locals():
    # Ensure FullDate is datetime
    df['FullDate'] = pd.to_datetime(df['FullDate'])
    
    # Extract Year for animation/selection
    df['Year'] = df['FullDate'].dt.year
    
    # Filter for valid years if needed (e.g., 1996-2006 as requested, though dataset usually ends earlier - we'll take all available)
    # The user mentioned 1996 to 2006. We'll ensure we use what's available.
    
    # Aggregate data: Group by Year, Customer, Employee, Date
    # We need Year in the grouping to use it for animation
    olap_df = df.groupby(['Year', 'CompanyName', 'LastName', 'FullDate']).size().reset_index(name='OrderCount')
    
    # Sort by Year to ensure animation plays correctly
    olap_df = olap_df.sort_values('Year')
    
    # Plotly Express 3D Scatter with Animation Frame
    fig_3d = px.scatter_3d(
        olap_df,
        x='CompanyName',
        y='LastName',
        z='FullDate',
        size='OrderCount',
        color='OrderCount',
        color_continuous_scale='Turbo',
        animation_frame='Year',
        animation_group='CompanyName',
        title='OLAP View: Orders by Customer, Employee, Date (Yearly Selection)',
        labels={'CompanyName': 'Customer', 'LastName': 'Employee', 'FullDate': 'Date'}
    )
    
    fig_3d.update_layout(
        scene=dict(
            xaxis_title='Customer',
            yaxis_title='Employee',
            zaxis_title='Date'
        ),
        height=800,
        margin=dict(r=0, l=0, b=0, t=50)
    )
    
    try:
        fig_3d.write_html("../figures/3d_orders_notebook.html")
        fig_3d.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 6. Delivery Status by Country

In [22]:
if 'df' in locals():
    delivery_by_country = df.groupby(['Country_x', 'DeliveredFlag']).size().reset_index(name='Count')
    delivery_by_country['Status'] = delivery_by_country['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig = px.bar(
        delivery_by_country,
        x='Country_x',
        y='Count',
        color='Status',
        title='Delivery Status by Country',
        barmode='stack',
        color_discrete_map={'Delivered': '#10b981', 'Not Delivered': '#f43f5e'}
    )
    
    fig.update_layout(
        xaxis_title='Country',
        yaxis_title='Number of Orders',
        height=600
    )
    
    try:
        fig.write_html("../figures/delivery_by_country_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 7. Comprehensive Dashboard

In [23]:
if 'df' in locals():
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Delivery Status', 'Orders by Country', 'Monthly Trend', 'Top Employees'),
        specs=[[{'type': 'pie'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )
    
    # Delivery Status Pie
    delivery_counts = df['DeliveredFlag'].value_counts()
    fig.add_trace(
        go.Pie(labels=['Delivered', 'Not Delivered'],
               values=[delivery_counts.get(1, 0), delivery_counts.get(0, 0)],
               marker=dict(colors=['#10b981', '#f43f5e'])),
        row=1, col=1
    )
    
    # Orders by Country
    country_orders = df['Country_x'].value_counts().head(5).reset_index()
    country_orders.columns = ['Country', 'Count']
    fig.add_trace(
        go.Bar(x=country_orders['Country'], y=country_orders['Count'],
               marker=dict(color='#06b6d4')),
        row=1, col=2
    )
    
    # Monthly Trend
    df['YearMonth'] = df['FullDate'].dt.to_period('M').astype(str)
    monthly = df.groupby('YearMonth').size().reset_index(name='Count')
    fig.add_trace(
        go.Scatter(x=monthly['YearMonth'], y=monthly['Count'],
                   mode='lines+markers', line=dict(color='#f43f5e')),
        row=2, col=1
    )
    
    # Top Employees
    df['EmployeeName'] = df['FirstName'] + ' ' + df['LastName']
    employee_orders = df['EmployeeName'].value_counts().head(5).reset_index()
    employee_orders.columns = ['Employee', 'Count']
    fig.add_trace(
        go.Bar(y=employee_orders['Employee'], x=employee_orders['Count'],
               orientation='h', marker=dict(color='#f59e0b')),
        row=2, col=2
    )
    
    fig.update_layout(
        title_text='Northwind Orders Dashboard',
        showlegend=False,
        height=900
    )
    
    try:
        fig.write_html("../figures/dashboard_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 4. Detailed Delivery Analysis
Analysis of delivery efficiency by Employee and Country.

In [24]:
# Delivery Status Analysis
if 'df' in locals():
    # 1. Delivery Status by Employee
    # Group by LastName and DeliveredFlag
    emp_delivery = df.groupby(['LastName', 'DeliveredFlag']).size().reset_index(name='Count')
    # Map flag to text
    emp_delivery['Status'] = emp_delivery['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig_emp = px.bar(emp_delivery, x='LastName', y='Count', color='Status', color_discrete_map={'Delivered': '#10b981', 'Not Delivered': '#f43f5e'}, 
                     title='Delivery Status by Employee', 
                     labels={'LastName': 'Employee', 'Count': 'Number of Orders'},
                     barmode='stack')
    
    # 2. Delivery Status by Country (Customer Country)
    # Note: Using Country_x based on previous cells (Customer Country)
    country_delivery = df.groupby(['Country_x', 'DeliveredFlag']).size().reset_index(name='Count')
    country_delivery['Status'] = country_delivery['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig_country = px.bar(country_delivery, x='Country_x', y='Count', color='Status', color_discrete_map={'Delivered': '#10b981', 'Not Delivered': '#f43f5e'}, 
                         title='Delivery Status by Country', 
                         labels={'Country_x': 'Country', 'Count': 'Number of Orders'},
                         barmode='stack')
    
    # Display figures
    fig_emp.update_layout(height=500)
    fig_country.update_layout(height=500)
    
    try:
        fig_emp.show()
        fig_country.show()
        fig_emp.write_html("../figures/delivery_by_employee.html")
        fig_country.write_html("../figures/delivery_by_country.html")
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")